# Create inscribed circles

This notebook creates a center and a radius of a maximum inscribed circle of each enclosed tessellation cell. 

It requires custom piece of code within pygeos. Therefore, we use a custom docker container based on `docker/Dockerfile`.

In [9]:
from dask.distributed import Client, LocalCluster, as_completed
from tqdm.notebook import tqdm
from time import time

In [6]:
client = Client(LocalCluster())
client

Client Scheduler: tcp://127.0.0.1:44843 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 134.91 GB


In [10]:
def _circle(chunk):
    import geopandas as gpd
    import pygeos
    gpd._compat.USE_PYGEOS = True
    
    s = time()
    df = gpd.read_parquet(f'../../urbangrammar_samba/spatial_signatures/tessellation/tess_{chunk}.pq', columns=['hindex', 'tessellation'])
    geoms = df.tessellation.values.data
    insc = pygeos.constructive.maximum_inscribed_circle(geoms, 0.1)
    lenghts = pygeos.length(insc)
    points = pygeos.points(pygeos.get_coordinates(insc)[::2])
    res = gpd.GeoDataFrame({'hindex': df.hindex, 'radius': lenghts}, geometry=points, crs=27700, index=df.index)
    res.to_parquet(f'../../urbangrammar_samba/spatial_signatures/inscribed_circle/circle_{chunk}.pq')
    return f"Chunk {chunk} processed sucessfully in {time() - s} seconds."

In [11]:
%%time
workers=16
inputs = iter(range(103))
futures = [client.submit(_circle, next(inputs)) for i in range(workers)]
ac = as_completed(futures)
for finished_future in ac:
    # submit new future 
    try:
        new_future = client.submit(_circle, next(inputs))
        ac.add(new_future)
    except StopIteration:
        pass
    print(finished_future.result())

Chunk 11 processed sucessfully in 75.18252611160278 seconds.
Chunk 3 processed sucessfully in 79.87826347351074 seconds.
Chunk 0 processed sucessfully in 86.3075304031372 seconds.
Chunk 2 processed sucessfully in 89.55268573760986 seconds.
Chunk 10 processed sucessfully in 90.74024438858032 seconds.
Chunk 1 processed sucessfully in 97.34478044509888 seconds.
Chunk 9 processed sucessfully in 100.90103268623352 seconds.
Chunk 15 processed sucessfully in 108.83816909790039 seconds.
Chunk 13 processed sucessfully in 111.28670144081116 seconds.
Chunk 7 processed sucessfully in 112.3213279247284 seconds.
Chunk 8 processed sucessfully in 115.32112979888916 seconds.
Chunk 14 processed sucessfully in 122.33609461784363 seconds.
Chunk 4 processed sucessfully in 127.47185468673706 seconds.
Chunk 17 processed sucessfully in 79.50266242027283 seconds.
Chunk 18 processed sucessfully in 74.23960661888123 seconds.
Chunk 16 processed sucessfully in 86.40599298477173 seconds.
Chunk 12 processed sucessfu